# 数据构造

In [ ]:
import json

prompt = """请根据后续提供的的文档片段，生成3组问题-答案对（可以根据文档片段以及问题得出答案），返回请以[{{"question":xx,"answer":xx}}]的格式。

以下是文档片段：
{}"""

token = 'f97b28aa71892d6d515007ef44ee5de6.WvLXD1MsFsGklOZa'

MODEL = "glm4-flash"

from zhipuai import ZhipuAI
client = ZhipuAI(api_key=token)


for line in open('heishenhua.txt','r', encoding='utf8'):
    messages = [{"role": "user", "content": prompt.format(line)}]
    try:
        response = client.chat.completions.create(
            model="glm-4-flash",
            messages=messages
        )
        result = response.choices[0].message.content
    except:
        continue
    fw = open('heishenhua_qa.txt', 'a', encoding='utf8')
    fw.write(json.dumps({'chunk':line, 'result':result}, ensure_ascii=False) + '\n')
    fw.close()

# 向量训练数据

In [ ]:
import json
data = []
for line in open('heishenhua_qa.txt',encoding='utf8'):
    line = json.loads(line)

    context = json.loads(line['chunk'])
    context = '标题:{}\n正文：{}'.format(context['title'],context['content'])

    try:
        result = json.loads(line['result'].lstrip('```json').rstrip('```'))
    except:
        
        result = [eval(i) for i in line['result'].split('\n') if i.startswith('[')]

    for item in result:
        if isinstance(item,list):
            item = item[0]
        data.append([context, item['question']])    


import pandas as pd
df = pd.DataFrame(columns=['context','query'],data=data)

df.drop_duplicates(subset=['query'],keep='first',inplace=True)
df.to_csv('embedding_data.csv',index=False)

# SFT训练数据

In [ ]:
import json
import random

data = []
contexts = []

for line in open('heishenhua_qa.txt',encoding='utf8'):
    line = json.loads(line)

    context = json.loads(line['chunk'])
    context = '标题:{}\n正文：{}'.format(context['title'],context['content'])

    try:
        result = json.loads(line['result'].lstrip('```json').rstrip('```'))
    except:
        
        result = [eval(i) for i in line['result'].split('\n') if i.startswith('[')]

    for item in result:
        if isinstance(item,list):
            item = item[0]
        data.append([context, item['question'],item['answer']])        
    contexts.append(context)


values = []
for item in data:
    item.append(random.sample(contexts,random.sample([4,5,6],1)[0]))
    if item[0] in item[-1]:
        item[-1].remove(item[0])

    if random.random() < 0.03:
        random.shuffle(item[-1])
        item.append('抱歉，我不知道')
    else:
        item[-1].append(item[0])  
        random.shuffle(item[-1])
        item.append(item[2])
    if len('\n'.join(item[-2])) < 10000:
        values.append(['\n'.join(item[-2]),item[1], item[-1]])
    
import pandas as pd
df = pd.DataFrame(columns=['context','query','answer'],data=values)
df.to_csv('sft_data.csv',index=False)